In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc,  font_manager
import matplotlib
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import statistics
matplotlib.rcParams['axes.unicode_minus'] = False
fontname = "C:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=fontname).get_name()
rc('font', family=font_name)

In [2]:
get_data = pd.read_csv('../data/TEMP/경락가격/양파/양파 월별 총거래(정제).csv', index_col=None)
# year_df = pd.read_csv('../data/TEMP/경락가격/오이/오이 년별 평균가격.csv', index_col=None)
# get_data = pd.read_csv('../data/TEMP/경락가격/오이/오이 월별 총거래(정제2).csv', index_col=None)

In [3]:
get_data['Date'] = pd.to_datetime(get_data['Date'])

In [4]:
years = get_data['Date'].dt.year.unique()

In [5]:

year_mean = []
year_addon = []
for i in years:
    for j in get_data.columns:
        if '평균' in j:
            print(get_data.loc[get_data['Date'].dt.year == i,'%s' %j][:5])
            data_p = get_data.loc[get_data['Date'].dt.year == i,'%s' %j].values
            year_addon.append(data_p.sum())
            # np_std= np.std(get_data.loc[get_data['Date'].dt.year == i,'%s' %j].values)
            # print('표준편차: ',np_std)
            # year_addon.append(np_std)
        elif j != 'Date':
            data_p = get_data.loc[get_data['Date'].dt.year == i,'%s' %j].values
            year_addon.append(data_p.sum())
        else:
            year_addon.append(i)
    year_mean.append(year_addon)
    year_addon = []

year_mean

0     850.299548
1    1065.910626
2    1184.568939
3     884.523227
4     694.235970
Name: 서울 평균거래금액, dtype: float64
0    801.968469
1    951.051331
2    877.264110
3    737.794945
4    582.040010
Name: 부산 평균거래금액, dtype: float64
0    793.571299
1    987.448199
2    855.629986
3    786.185141
4    553.094395
Name: 대구 평균거래금액, dtype: float64
0    807.809883
1    993.530845
2    988.243598
3    863.005288
4    681.568347
Name: 대전 평균거래금액, dtype: float64
0    772.806841
1    784.575261
2    798.343330
3    780.159375
4    641.744594
Name: 광주 평균거래금액, dtype: float64
12    1576.603389
13    1773.265723
14    1348.503606
15     857.397660
16     548.205464
Name: 서울 평균거래금액, dtype: float64
12    1437.990074
13    1644.517116
14    1435.160047
15     701.833985
16     475.627423
Name: 부산 평균거래금액, dtype: float64
12    1512.716622
13    1698.810012
14    1518.189287
15     714.078581
16     510.904832
Name: 대구 평균거래금액, dtype: float64
12    1521.749864
13    1750.827003
14    1520.229328
15     801.9497

[[2020,
  28643863.0,
  23410318366,
  11420.180675475063,
  14674616.399999999,
  12034439360,
  9960.323091937185,
  35959296.2,
  28489421650,
  9904.703498106268,
  40277201.5,
  34538198720,
  10529.200349497716,
  8349609.0,
  6495632730,
  9895.217795726934],
 [2021,
  35038498.8,
  28286384615,
  12986.592743777615,
  10983886.0,
  9289969200,
  10393.976760225709,
  31075443.800000004,
  26676852760,
  10389.179751867028,
  39113825.5,
  32464919992,
  10989.206760225354,
  11895149.0,
  9602812040,
  10035.454593104858],
 [2022,
  29486508.7,
  20710385185,
  13058.93215428937,
  13677987.6,
  12150281500,
  10585.795208854135,
  43079801.8,
  32334159800,
  10147.022005332825,
  37665097.0,
  31121738576,
  10604.207482075259,
  12633129.700000001,
  8802605004,
  10513.49781721824],
 [2023,
  19939413.2,
  22819660100,
  10353.634732921853,
  7341655.8,
  7523010890,
  8431.98851855311,
  26685362.9,
  27462259900,
  8578.25053156864,
  21958250.0,
  24399853320,
  9043.703

In [6]:
year_df = pd.DataFrame(year_mean, columns=['year', '서울 총거래물량', '서울 총거래금액', '서울 평균단가','부산 총거래물량', '부산 총거래금액', '부산 평균단가', '대구 총거래물량', '대구 총거래금액', '대구 평균단가', '대전 총거래물량', '대전 총거래금액', '대전 평균단가', '광주 총거래물량', '광주 총거래금액', '광주 평균단가'])

In [7]:
for i in years:
    year_df.loc[year_df['year']== i,'서울 평균단가'] = year_df['서울 총거래금액']/year_df['서울 총거래물량']
    year_df.loc[year_df['year']== i,'부산 평균단가'] = year_df['부산 총거래금액']/year_df['부산 총거래물량']
    year_df.loc[year_df['year']== i,'대구 평균단가'] = year_df['대구 총거래금액']/year_df['대구 총거래물량']
    year_df.loc[year_df['year']== i,'대전 평균단가'] = year_df['대전 총거래금액']/year_df['대전 총거래물량']
    year_df.loc[year_df['year']== i,'광주 평균단가'] = year_df['광주 총거래금액']/year_df['광주 총거래물량']

year_df

,year,서울 총거래물량,서울 총거래금액,서울 평균단가,부산 총거래물량,부산 총거래금액,부산 평균단가,대구 총거래물량,대구 총거래금액,대구 평균단가,대전 총거래물량,대전 총거래금액,대전 평균단가,광주 총거래물량,광주 총거래금액,광주 평균단가
0,2020,28643863.0,23410318366,817.289147,14674616.4,12034439360,820.085448,35959296.2,28489421650,792.268611,40277201.5,34538198720,857.512375,8349609.0,6495632730,777.956516
1,2021,35038498.8,28286384615,807.294421,10983886.0,9289969200,845.781648,31075443.8,26676852760,858.454442,39113825.5,32464919992,830.011373,11895149.0,9602812040,807.288084
2,2022,29486508.7,20710385185,702.368171,13677987.6,12150281500,888.309147,43079801.8,32334159800,750.564266,37665097.0,31121738576,826.275280,12633129.7,8802605004,696.787353
3,2023,19939413.2,22819660100,1144.449933,7341655.8,7523010890,1024.702205,26685362.9,27462259900,1029.113226,21958250.0,24399853320,1111.192983,9875231.5,9598267980,971.953719


In [8]:
get_data = get_data.rename(columns={'서울 평균거래금액':'서울 평균단가', '부산 평균거래금액':'부산 평균단가', '대구 평균거래금액':'대구 평균단가', '대전 평균거래금액':'대전 평균단가', '광주 평균거래금액':'광주 평균단가'})

In [9]:
year_df['전국 평균단가'] = (year_df['서울 총거래금액'] + year_df['부산 총거래금액'] + year_df['대구 총거래금액'] + year_df['대전 총거래금액'] + year_df['광주 총거래금액']) / (year_df['서울 총거래물량'] + year_df['부산 총거래물량'] + year_df['대구 총거래물량'] + year_df['대전 총거래물량'] + year_df['광주 총거래물량']) 

In [10]:
year_df.to_csv('../data/TEMP/경락가격/양파/양파 년별 평균가격.csv', index=False, encoding='UTF-8')

In [11]:
year_df

,year,서울 총거래물량,서울 총거래금액,서울 평균단가,부산 총거래물량,부산 총거래금액,부산 평균단가,대구 총거래물량,대구 총거래금액,대구 평균단가,대전 총거래물량,대전 총거래금액,대전 평균단가,광주 총거래물량,광주 총거래금액,광주 평균단가,전국 평균단가
0,2020,28643863.0,23410318366,817.289147,14674616.4,12034439360,820.085448,35959296.2,28489421650,792.268611,40277201.5,34538198720,857.512375,8349609.0,6495632730,777.956516,820.674333
1,2021,35038498.8,28286384615,807.294421,10983886.0,9289969200,845.781648,31075443.8,26676852760,858.454442,39113825.5,32464919992,830.011373,11895149.0,9602812040,807.288084,829.939832
2,2022,29486508.7,20710385185,702.368171,13677987.6,12150281500,888.309147,43079801.8,32334159800,750.564266,37665097.0,31121738576,826.275280,12633129.7,8802605004,696.787353,769.863969
3,2023,19939413.2,22819660100,1144.449933,7341655.8,7523010890,1024.702205,26685362.9,27462259900,1029.113226,21958250.0,24399853320,1111.192983,9875231.5,9598267980,971.953719,1069.966723


In [12]:
years = get_data['Date'].dt.year.unique()
months = get_data['Date'].dt.month.unique()
grade_data = []
for i in years:
    for j in months:
        if ( i != 2023 ) or ( j not in [9, 10, 11, 12]):
            g_date = str(i)+"-"+str(j)
            seoul_data = year_df.loc[year_df['year']==i, '서울 평균단가'].values
            seoul_data1 = get_data.loc[(get_data['Date'].dt.month == j) & (get_data['Date'].dt.year==i), '서울 평균단가'].values
            seoul_grade = seoul_data - seoul_data1

            busan_data = year_df.loc[year_df['year']==i, '부산 평균단가'].values
            busan_data1 = get_data.loc[(get_data['Date'].dt.month == j) & (get_data['Date'].dt.year==i), '부산 평균단가'].values
            busan_grade = busan_data - busan_data1

            daegu_data = year_df.loc[year_df['year']==i, '대구 평균단가'].values
            daegu_data1 = get_data.loc[(get_data['Date'].dt.month == j) & (get_data['Date'].dt.year==i), '대구 평균단가'].values
            daegu_grade = daegu_data - daegu_data1

            daejeon_data = year_df.loc[year_df['year']==i, '대전 평균단가'].values
            daejeon_data1 = get_data.loc[(get_data['Date'].dt.month == j) & (get_data['Date'].dt.year==i), '광주 평균단가'].values
            daejeon_grade = daejeon_data - daejeon_data1

            gwangju_data = year_df.loc[year_df['year']==i, '광주 평균단가'].values
            gwangju_data1 = get_data.loc[(get_data['Date'].dt.month == j) & (get_data['Date'].dt.year==i), '광주 평균단가'].values
            gwangju_grade = gwangju_data - gwangju_data1
            print(len(gwangju_grade))
            grade_data.append([g_date,seoul_grade[0], busan_grade[0], daegu_grade[0], daejeon_grade[0], gwangju_grade[0]])
print(grade_data)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
[['2020-1', -33.01040046925948, 18.11697892647146, -1.3026881733371738, 84.70553430798554, 5.149675511201281], ['2020-2', -248.62147844153844, -130.96588357878318, -195.1795882902892, 72.93711427918515, -6.6187445175991115], ['2020-3', -367.2797921035618, -57.17866258927768, -63.361375392007744, 59.16904486164469, -20.38681393513957], ['2020-4', -67.23407969907566, 82.2905022497547, 6.083470126533371, 77.3529996682604, -2.202859128523869], ['2020-5', 123.05317718038395, 238.04543818087006, 239.17421594176244, 215.76778102722278, 136.21192223043852], ['2020-6', 233.70150827630266, 313.0861202149631, 308.71012811306565, 369.91365377911484, 290.3577949823306], ['2020-7', 138.21160523381582, 196.76702403459058, 204.78079762842788, 254.0459223442682, 174.49006354748394], ['2020-8', -148.6027903241544, 11.644082022075622, 9.839351753745518, 41.014741547781, -38.541117249003264], ['2020-9', -218.1902137457

In [13]:
mean_data = pd.DataFrame(grade_data, columns=['Date','서울 평균단가 편차', '부산 평균단가 편차', '대구 평균단가 편차', '대전 평균단가 편차', '광주 평균단가 편차'])

In [14]:
mean_data['Date'] = pd.to_datetime(mean_data['Date'])

In [15]:
mean_all = pd.merge(get_data, mean_data, on='Date', how='inner')

In [16]:
mean_all = mean_all[['Date', '서울 총거래물량', '서울 총거래금액', '서울 평균단가', '서울 평균단가 편차','부산 총거래물량', '부산 총거래금액', '부산 평균단가', '부산 평균단가 편차', '대구 총거래물량', '대구 총거래금액', '대구 평균단가', '대구 평균단가 편차','대전 총거래물량', '대전 총거래금액', '대전 평균단가', '대전 평균단가 편차', '광주 총거래물량', '광주 총거래금액', '광주 평균단가', '광주 평균단가 편차']]

In [17]:
mean_all['전국 평균단가'] = (mean_all['서울 총거래금액'] + mean_all['부산 총거래금액'] + mean_all['대구 총거래금액'] + mean_all['대전 총거래금액'] + mean_all['광주 총거래금액']) / (mean_all['서울 총거래물량'] + mean_all['부산 총거래물량'] + mean_all['대구 총거래물량'] + mean_all['대전 총거래물량'] + mean_all['광주 총거래물량']) 

In [18]:
mean_all.head()

,Date,서울 총거래물량,서울 총거래금액,서울 평균단가,서울 평균단가 편차,부산 총거래물량,부산 총거래금액,부산 평균단가,부산 평균단가 편차,대구 총거래물량,...,대구 평균단가 편차,대전 총거래물량,대전 총거래금액,대전 평균단가,대전 평균단가 편차,광주 총거래물량,광주 총거래금액,광주 평균단가,광주 평균단가 편차,전국 평균단가
0,2020-01-01,196129.0,166768400,850.299548,-33.010400,1242260.0,996253350,801.968469,18.116979,2276705.7,...,-1.302688,2946856.0,2380499400,807.809883,84.705534,148872.0,115049300,772.806841,5.149676,802.443257
1,2020-02-01,161702.0,172359880,1065.910626,-248.621478,1262447.0,1200651900,951.051331,-130.965884,2582537.8,...,-195.179588,2820738.0,2802490210,993.530845,72.937114,40260.0,31587000,784.575261,-6.618745,983.914010
2,2020-03-01,174813.0,207078050,1184.568939,-367.279792,1199742.8,1052491300,877.264110,-57.178663,2244144.0,...,-63.361375,2691912.0,2660264800,988.243598,59.169045,504657.0,402889550,798.343330,-20.386814,916.013848
3,2020-04-01,10802310.0,9554894100,884.523227,-67.234080,356549.0,263060050,737.794945,82.290502,1166191.4,...,6.083470,4174226.0,3602379110,863.005288,77.353000,1195291.0,932517480,780.159375,-2.202859,862.959390
4,2020-05-01,12249069.0,8503744300,694.235970,123.053177,803859.0,467878100,582.040010,238.045438,996047.7,...,239.174216,4766306.0,3248563300,681.568347,215.767781,1391894.0,893240450,641.744594,136.211922,676.211993


In [19]:
mean_all.to_csv('../data/TEMP/경락가격/양파/양파 월별 데이터와 평균편차.csv', index=False, encoding='UTF-8')

In [20]:
year_df = pd.read_csv('../data/TEMP/경락가격/양파/양파 년별 평균가격.csv', index_col=None)
get_data = pd.read_csv('../data/TEMP/경락가격/양파/양파 월별 데이터와 평균편차.csv', index_col=None)

In [21]:
get_data['Date'] = pd.to_datetime(get_data['Date'])

In [22]:
years = get_data['Date'].dt.year.unique()
months = get_data['Date'].dt.month.unique()
all_grade_data = []
for i in years:
    for j in months:
        if ( i != 2023 ) or ( j not in [9, 10, 11, 12]):
            g_date = str(i)+"-"+str(j)
            total_data = year_df.loc[year_df['year']==i, '전국 평균단가'].values
            total_data1 = get_data.loc[(get_data['Date'].dt.month == j) & (get_data['Date'].dt.year==i), '전국 평균단가'].values
            total_grade = total_data - total_data1

            all_grade_data.append([g_date,total_grade[0]])
print(all_grade_data)

[['2020-1', 18.231075571573], ['2020-2', -163.23967732001609], ['2020-3', -95.3395156625113], ['2020-4', -42.28505747359509], ['2020-5', 144.46233968506863], ['2020-6', 319.32486470479995], ['2020-7', 210.94221852323665], ['2020-8', -4.163188017610764], ['2020-9', -101.76767972309153], ['2020-10', -208.5362103326039], ['2020-11', -285.3832718283426], ['2020-12', -290.74624709305897], ['2021-1', -670.9831682535272], ['2021-2', -867.0614585847228], ['2021-3', -596.8982133297001], ['2021-4', 5.659269047051566], ['2021-5', 294.29773306286654], ['2021-6', 316.7540413308959], ['2021-7', 296.4490261351542], ['2021-8', 127.53765414883492], ['2021-9', 6.809180924593761], ['2021-10', 12.525035871313435], ['2021-11', 35.70141529707507], ['2021-12', 185.26663126666153], ['2022-1', 261.63275823821897], ['2022-2', 368.27670920851034], ['2022-3', 442.9715342385535], ['2022-4', 261.16528362925027], ['2022-5', 232.79365332403268], ['2022-6', -166.0543457344337], ['2022-7', -325.39299450974215], ['2022-

In [23]:
total_df = pd.DataFrame(all_grade_data, columns=['Date', '전국 평균단가 편차'])

In [24]:
total_df['Date'] = pd.to_datetime(total_df['Date'])

In [25]:
total_all = pd.merge(get_data, total_df, on='Date', how='inner')

In [26]:
total_all.to_csv('../data/TEMP/경락가격/양파/양파 월별 데이터와 평균편차(전국포함).csv', index=False, encoding='UTF-8')